In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import OUTPUT_PATH, Version, FuncGroups, CollisionType, VERSION_DEFAULT


def count_dup(v: Version):
    functions = FuncGroups.from_dump(v.dwarf_funcs_path)
    results = {c: FuncGroups() for c in CollisionType}

    for name, group in functions.iter_groups():
        results[group.get_collision_type()].add_group(name, group)

    for c in CollisionType:
        results[c].save_groups(OUTPUT_PATH / "dup" / v.name / f"{c.name.lower()}.txt")

    return {k: v.num_groups for k, v in results.items()}


# count_dup(VERSION_DEFAULT)

In [5]:
from depsurf import Versions, VersionGroups
from utils import save_df

df = VersionGroups(Versions.REGULAR).apply(count_dup)
save_df(df, "dup")

[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Downloads/bpf-study/data/dwarf_funcs/4.4.0-21-generic-amd64.jsonl
[       groups.py:56 ] INFO: Saved FuncGroups(17237 groups, 17237 functions) to /Users/szhong/Downloads/bpf-study/output/dup/4.4.0-21-generic-amd64/unique_global.txt
[       groups.py:56 ] INFO: Saved FuncGroups(35710 groups, 35710 functions) to /Users/szhong/Downloads/bpf-study/output/dup/4.4.0-21-generic-amd64/unique_static.txt
[       groups.py:56 ] INFO: Saved FuncGroups(4042 groups, 53294 functions) to /Users/szhong/Downloads/bpf-study/output/dup/4.4.0-21-generic-amd64/include_dup.txt
[       groups.py:56 ] INFO: Saved FuncGroups(404 groups, 961 functions) to /Users/szhong/Downloads/bpf-study/output/dup/4.4.0-21-generic-amd64/static_static.txt
[       groups.py:56 ] INFO: Saved FuncGroups(10 groups, 20 functions) to /Users/szhong/Downloads/bpf-study/output/dup/4.4.0-21-generic-amd64/static_global.txt
[       groups.py:68 ] INFO: Loading funcs from /Users/

Unique Global  Unique Static  Static Dup. with #include  Static-Static Collision  Static-Global Collision
Regular 4.4.0-21-generic-amd64           17237          35710                       4042                      404                       10
        4.8.0-22-generic-amd64           18121          37504                       4329                      401                       26
        4.10.0-19-generic-amd64          18573          38612                       4430                      397                       24
        4.13.0-16-generic-amd64          19318          40389                       4668                      398                       25
        4.15.0-20-generic-amd64          20098          41740                       4809                      398                       26
        4.18.0-10-generic-amd64          21194          44004                       5014                      411                       26
        5.0.0-13-generic-amd64           21644          45077                       5153                      408                       26
        5.3.0-18-generic-amd64           22336          46955                       5348                      408                       26
        5.4.0-26-generic-amd64           22713          48193                       5461                      411                       27
        5.8.0-25-generic-amd64           23500          49973                       5756                      421                       26
        5.11.0-16-generic-amd64          25893          51626                       5907                      438                       26
        5.13.0-19-generic-amd64          26296          52395                       6036                      439                       26
        5.15.0-25-generic-amd64          26642          53331                       6162                      444                       26
        5.19.0-21-generic-amd64          27514          55005                       6599                      453                       29
        6.2.0-20-generic-amd64           28339          56343                       6876                      458                       32
        6.5.0-9-generic-amd64            29831          58122                       7079                      481                       30
        6.8.0-31-generic-amd64           31543          60185                       7418                      498                       29

In [4]:
from utils import load_df
from depsurf import Versions, TAB_PATH


def make_col(v):
    return ("Linux Kernel Version", r"\multicolumn{1}{c}{" + v.short_version + "}")


df = load_df("dup")
df = df.loc[("Regular", list(Versions.LTS)), :].T
df.columns = df.columns.map(lambda x: make_col(x[1]))
df = df.map(lambda v: f"{v / 1000:.1f}k" if v > 1000 else v)
with open(TAB_PATH / "dup.tex", "w") as f:
    latex = df.to_latex(multicolumn_format="c", column_format="l|rrrrr")
    latex = latex.replace("#", "\\#")
    f.write(latex)
df

[           pd.py:53 ] INFO: Loaded df from /Users/szhong/Downloads/bpf-study/output/dup.pkl


Linux Kernel Version                                                                                                  
                          \multicolumn{1}{c}{4.4} \multicolumn{1}{c}{4.15} \multicolumn{1}{c}{5.4} \multicolumn{1}{c}{5.15} \multicolumn{1}{c}{6.8}
Unique Global                               17.2k                    20.1k                   22.7k                    26.6k                   31.5k
Unique Static                               35.7k                    41.7k                   48.2k                    53.3k                   60.2k
Static Dup. with #include                    4.0k                     4.8k                    5.5k                     6.2k                    7.4k
Static-Static Collision                       404                      398                     411                      444                     498
Static-Global Collision                        10                       26                      27                       26                      29